In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
#загрузка тренировочной выборки
salary = pd.read_csv('C:/Tomato/salary-train.csv')
salary.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [3]:
#опустить все до нижнего регистра и убрать все посторонние знаки кроме букв и цифр
salary = salary.apply(lambda x: x.astype(str).str.lower())

In [4]:
salary.replace('[^a-zA-Z0-9]',' ',regex = True)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000
2,online content and brand manager luxury reta...,south east london,permanent,40000
3,a great local marketleader is seeking a perman...,dereham,permanent,22500
4,registered nurse rgn nursing home for young...,sutton coldfield,nan,20355
5,sales and marketing assistant will provide adm...,crawley,nan,22500
6,vacancy ladieswear fashion area manager regi...,uk,permanent,32000
7,reference lr jan our client is one of th...,bristol,permanent,30000
8,sponsorship manager london the company a marke...,central london,permanent,31500
9,about barclays barclays moves lends invests ...,south east london,permanent,42499


In [5]:
#преобразуем текст в вектор
vect = TfidfVectorizer(min_df = 5)
X = vect.fit_transform(salary['FullDescription'])

In [6]:
#пропуски заменяем на нан
salary['LocationNormalized'].fillna('nan',inplace = True)
salary['ContractTime'].fillna('nan', inplace = True)

In [7]:
#one-hot кодирование признаков
emc = DictVectorizer()
X_train_categ = emc.fit_transform( 
                salary[['LocationNormalized','ContractTime']]
                .to_dict('records'))

In [8]:
#кормим и учим нашу модель этим говном
vect = TfidfVectorizer(min_df = 5)
X = vect.fit_transform(salary['FullDescription'])
salary['LocationNormalized'].fillna('nan',inplace = True)
salary['ContractTime'].fillna('nan', inplace = True)
emc = DictVectorizer()
X_train_categ = emc.fit_transform( 
                salary[['LocationNormalized','ContractTime']]
                .to_dict('records'))
#объединяем разреженные матрицы: объекты + признаки
from scipy.sparse import hstack
X2_train = hstack([X, X_train_categ])
clf = Ridge(alpha = 1)
clf.fit(X2_train, salary['SalaryNormalized'])


Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [12]:
#загружаем тестовую выборку и проделываем ту же шляпу,
#только Х1 уже не кормим, а чисто трансформируем по предыдущему примеру
forecast = pd.read_csv('C:/Tomato/salary-test-mini.csv')
forecast = forecast.apply(lambda x: x.astype(str).str.lower())
forecast.replace('[^a-zA-Z0-9]',' ',regex = True)

X1 = vect.transform(forecast['FullDescription'])
forecast['LocationNormalized'].fillna('nan',inplace = True)
forecast['ContractTime'].fillna('nan', inplace = True)
X_test_categ = emc.transform(
                forecast[['LocationNormalized','ContractTime']]
                           .to_dict('records'))
X2_test = hstack([X1, X_test_categ])
#хотим предсказать, но оп! и гавно
#clf = Ridge(alpha=1, random_state = 241)
y = clf.predict(X2_test)
print(np.round(y,2))

[56545.54 37186.49]
